In [ ]:
# --- Cell 1: Tải các thư viện ---
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
print(f"TensorFlow version: {tf.__version__}")

In [ ]:
# --- Cell 2 (train.ipynb - SỬA LỖI CHO CSV): Tải dữ liệu đã xử lý ---
file_name = 'kpi_processed.csv' # <-- Đã đổi tên file
try:
    # *** THAY ĐỔI QUAN TRỌNG ***
    # Chúng ta dùng pd.read_csv và thêm 2 tham số:
    # 1. index_col=0: Báo cho Pandas rằng cột đầu tiên (cột 0) là index.
    # 2. parse_dates=True: Báo cho Pandas chuyển đổi index đó thành Datetime.
    df = pd.read_csv(file_name, index_col=0, parse_dates=True)
    
    print(f"Tải file '{file_name}' thành công.")
    print(f"Dữ liệu có {len(df)} hàng, kéo dài từ {df.index.min()} đến {df.index.max()}.")
    print("\nThông tin dữ liệu (df.info()):")
    df.info() 
    
except Exception as e:
    print(f"LỖI: Không thể tải file '{file_name}'.")
    print(e)

In [ ]:

# --- Cell 3: Xử lý dữ liệu (Quan trọng!) ---
# Mục tiêu: Đảm bảo MỌI cell_name đều có đầy đủ 15 phút, không bị ngắt quãng.
# Nếu thiếu, chúng ta sẽ lấp đầy bằng 0.
if 'df' in locals():
    # Xác định các cột KPI để dự đoán
    FEATURE_COLS = ['ps_traffic_mb', 'avg_rrc_connected_user', 'prb_dl_used', 'prb_dl_available_total', 'prb_utilization']
    N_FEATURES = len(FEATURE_COLS)
    print(f"Dự đoán {N_FEATURES} features: {FEATURE_COLS}")
    # Tạo ra một DataFrame đầy đủ cho MỌI cell
    print("Đang xử lý: Đảm bảo dữ liệu 15 phút đầy đủ cho mỗi cell...")
    # Lấy tất cả các cell độc nhất
    all_cells = df['cell_name'].unique()
    # Tạo ra một index thời gian 15 phút đầy đủ từ đầu đến cuối
    # (Sửa lỗi cảnh báo: '15T' -> '15min')
    full_time_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='15min')
    # Tạo MultiIndex (cell_name, timestamp)
    multi_index = pd.MultiIndex.from_product([all_cells, full_time_index], names=['cell_name', 'timestamp'])
    # Lấy các cột dữ liệu
    df_data_only = df[FEATURE_COLS + ['cell_name']]
    # Đưa timestamp ra thành một cột
    df_data_only = df_data_only.reset_index()
    # Xử lý các (cell_name, timestamp) bị trùng lặp
    print(f"Dữ liệu trước khi xử lý trùng lặp: {len(df_data_only)} hàng")
    df_grouped_unique = df_data_only.groupby(['cell_name', 'timestamp']).mean()
    print(f"Dữ liệu sau khi xử lý trùng lặp (groupby.mean): {len(df_grouped_unique)} hàng")
    # Reindex với MultiIndex đầy đủ, điền 0 vào các chỗ thiếu
    df_full = df_grouped_unique.reindex(multi_index, fill_value=0)
    # *** (BƯỚC SỬA LỖI KEYERROR) ***
    # 1. Reset TẤT CẢ các cấp index ('cell_name', 'timestamp') ra thành cột
    df_full = df_full.reset_index() 
    # 2. Đặt cột 'timestamp' làm index mới.
    # 'cell_name' bây giờ sẽ tự động là một cột, chính xác như chúng ta muốn.
    df_full = df_full.set_index('timestamp')
    # *** (KẾT THÚC SỬA LỖI) ***
    print(f"Dữ liệu gốc có {len(df)} hàng.")
    print(f"Dữ liệu đã điền đầy đủ (sau reindex) có {len(df_full)} hàng.")
    print("Hoàn tất xử lý.")
    print("\n5 dòng đầu của dữ liệu đã xử lý đầy đủ:")
    print(df_full.head())
else:
    print("Lỗi: DataFrame 'df' không tồn tại. Vui lòng chạy lại Cell 2.")

In [ ]:
# --- Cell 4: 
# Data có tần suất 15 phút (4 mẫu/giờ)
TIMESTEPS_PER_HOUR = 4
TIMESTEPS_PER_DAY = 24 * TIMESTEPS_PER_HOUR # 96
# Input: 2 ngày
INPUT_DAYS = 2
INPUT_STEPS = INPUT_DAYS * TIMESTEPS_PER_DAY # 2 * 96 = 192
# Output: 1 ngày
OUTPUT_DAYS = 1
OUTPUT_STEPS = OUTPUT_DAYS * TIMESTEPS_PER_DAY # 1 * 96 = 96

print(f"Cấu hình mô hình: Input {INPUT_STEPS} bước (2 ngày), Output {OUTPUT_STEPS} bước (1 ngày).")
print(f"Yêu cầu dữ liệu tối thiểu cho 1 mẫu: 276 bước (28 ngày)")

# TÁCH DỮ LIỆU THEO THỜI GIAN (70/20/10)
if 'df_full' in locals():
    total_duration = df_full.index.max() - df_full.index.min()
    # 70% cho Train
    train_end_time = df_full.index.min() + total_duration * 0.7
    # Thêm 20% cho Validation (tổng 90%)
    val_end_time = df_full.index.min() + total_duration * 0.9
    # 10% còn lại cho Test
    train_df = df_full[df_full.index < train_end_time]
    val_df = df_full[(df_full.index >= train_end_time) & (df_full.index < val_end_time)]
    test_df = df_full[df_full.index >= val_end_time]
    print(f"\n Train: {train_df.shape}")
    print(f"Val:   {val_df.shape}")
    print(f"Test:  {test_df.shape}")
else:
    print("Lỗi: DataFrame 'df_full' không tồn tại. Vui lòng chạy lại Cell 3.")

In [ ]:

# --- Cell 5: Chuẩn hóa (Scaling) Dữ liệu ---
# Chúng ta phải fit Scaler CHỈ trên dữ liệu train để tránh rò rỉ dữ liệu

# Kiểm tra xem có NaN hay Inf trong train_df không
print("Kiểm tra dữ liệu bẩn:")
print("Số lượng NaN:", train_df.isna().sum().sum())
print("Số lượng Inf:", np.isinf(train_df[FEATURE_COLS].values).sum())

# Xử lý thay thế (nếu có)
train_df = train_df.replace([np.inf, -np.inf], 0)
train_df = train_df.fillna(0)

val_df = val_df.replace([np.inf, -np.inf], 0)
val_df = val_df.fillna(0)

test_df = test_df.replace([np.inf, -np.inf], 0)
test_df = test_df.fillna(0)

print("Đã làm sạch dữ liệu NaN/Inf.")
if 'train_df' in locals():
    # Khởi tạo Scaler
    scaler = MinMaxScaler()
    
    # 1. Fit scaler CHỈ trên dữ liệu train (chỉ các cột features)
    # Chúng ta phải fit trên toàn bộ dữ liệu train để scaler học được min/max
    scaler.fit(train_df[FEATURE_COLS])
    
    # 2. Transform cả 3 tập
    # Lưu lại 'cell_name' để dùng cho việc nhóm
    train_cells = train_df['cell_name']
    val_cells = val_df['cell_name']
    test_cells = test_df['cell_name']

    # Transform
    train_scaled_data = scaler.transform(train_df[FEATURE_COLS])
    val_scaled_data = scaler.transform(val_df[FEATURE_COLS])
    test_scaled_data = scaler.transform(test_df[FEATURE_COLS])
    
    # 3. Tạo lại DataFrame đã scale (việc này giúp nhóm dễ dàng hơn)
    scaled_train_df = pd.DataFrame(train_scaled_data, columns=FEATURE_COLS, index=train_df.index)
    scaled_train_df['cell_name'] = train_cells

    scaled_val_df = pd.DataFrame(val_scaled_data, columns=FEATURE_COLS, index=val_df.index)
    scaled_val_df['cell_name'] = val_cells

    scaled_test_df = pd.DataFrame(test_scaled_data, columns=FEATURE_COLS, index=test_df.index)
    scaled_test_df['cell_name'] = test_cells
    
    print("Hoàn tất scaling dữ liệu.")
    print("Dữ liệu train sau khi scale (5 dòng đầu):")
    print(scaled_train_df.head())

# %%

In [ ]:

# --- Cell 6: Hàm tạo cửa sổ (Windowing) ---
# Đây là hàm quan trọng nhất:
# Nó sẽ duyệt qua TỪNG cell_name, sau đó tạo các cặp (X, y)
# X = 672 bước (7 ngày), y = 96 bước (1 ngày)

def create_windows(data_df, input_steps, output_steps, feature_cols):
    """
    Tạo các cửa sổ X (input) và y (output) từ dữ liệu đã scale,
    nhóm theo 'cell_name'.
    """
    X, y = [], []
    
    # Nhóm dữ liệu theo từng cell
    grouped = data_df.groupby('cell_name')
    
    total_cells = len(grouped)
    print(f"Bắt đầu tạo cửa sổ cho {total_cells} cell...")
    
    cell_count = 0
    for cell_id, cell_data in grouped:
        cell_count += 1
        if cell_count % 50 == 0:
            print(f"  ...đang xử lý cell {cell_count}/{total_cells} (ID: {cell_id})")
            
        # Lấy dữ liệu số của cell này
        cell_features = cell_data[feature_cols].values
        
        # Tổng số mẫu của cell này
        total_samples = len(cell_features)
        
        # Tổng độ dài cần thiết cho 1 cửa sổ
        total_window_len = input_steps + output_steps
        
        # Trượt cửa sổ
        for i in range(total_samples - total_window_len + 1):
            # i là điểm bắt đầu của input
            input_start = i
            input_end = i + input_steps
            
            # output_end là điểm kết thúc của output
            output_end = input_end + output_steps
            
            # Lấy cửa sổ X và y
            window_X = cell_features[input_start:input_end, :]
            window_y = cell_features[input_end:output_end, :]
            
            X.append(window_X)
            y.append(window_y)
            
    print(f"Hoàn tất tạo cửa sổ. Đã tạo {len(X)} mẫu.")
    
    # Chuyển list thành Numpy array
    return np.array(X), np.array(y)

# %%

In [ ]:
# --- Cell 7: Áp dụng hàm Windowing ---

if 'scaled_train_df' in locals():
    print("--- Đang tạo mẫu Train (X_train, y_train) ---")
    X_train, y_train = create_windows(scaled_train_df, INPUT_STEPS, OUTPUT_STEPS, FEATURE_COLS)
    
    print("\n--- Đang tạo mẫu Validation (X_val, y_val) ---")
    X_val, y_val = create_windows(scaled_val_df, INPUT_STEPS, OUTPUT_STEPS, FEATURE_COLS)
    
    print("\n--- Đang tạo mẫu Test (X_test, y_test) ---")
    X_test, y_test = create_windows(scaled_test_df, INPUT_STEPS, OUTPUT_STEPS, FEATURE_COLS)
    
    print("\n--- Kích thước dữ liệu (Shape) ---")
    print(f"X_train shape: {X_train.shape}") # (Số mẫu, 672, N_FEATURES)
    print(f"y_train shape: {y_train.shape}") # (Số mẫu, 96, N_FEATURES)
    print(f"X_val shape:   {X_val.shape}")
    print(f"y_val shape:   {y_val.shape}")
    print(f"X_test shape:  {X_test.shape}")
    print(f"y_test shape:  {y_test.shape}")
else:
    print("Lỗi: Dữ liệu đã scale không tồn tại. Vui lòng chạy lại Cell 5.")
    

In [ ]:
# --- Cell 8 (Cập nhật): Xây dựng mô hình LSTM (Seq2Seq) ---
# Cập nhật Input (2016 steps) và Output (672 steps)
# --- SỬA LẠI CELL 8 ---
if 'X_train' in locals() and X_train.shape[0] > 0:
    keras.backend.clear_session()
    N_UNITS_LSTM = 100 

    model = Sequential()
    
    # Input
    model.add(Input(shape=(INPUT_STEPS, N_FEATURES)))
    
    # Encoder: ĐỔI 'relu' THÀNH 'tanh'
    model.add(LSTM(N_UNITS_LSTM, activation='tanh')) 
    
    # Repeater
    model.add(RepeatVector(OUTPUT_STEPS))
    
    # Decoder: ĐỔI 'relu' THÀNH 'tanh'
    model.add(LSTM(N_UNITS_LSTM, activation='tanh', return_sequences=True))
    
    # Output Layer
    model.add(TimeDistributed(Dense(N_FEATURES)))
    
    # --- QUAN TRỌNG: Cấu hình Optimizer để kẹp Gradient (Clipnorm) ---
    # Nếu gradient vẫn quá lớn, clipnorm sẽ cắt bớt nó lại
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)
    
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    model.summary()

    
else:
    print("Lỗi: Không có dữ liệu training (X_train) để xây dựng mô hình.")

In [ ]:

# %%
# --- Cell 9: Huấn luyện mô hình ---

if 'model' in locals():
    print("Bắt đầu huấn luyện (training) mô hình...")
    
    # Hằng số huấn luyện
    EPOCHS = 20 # Số lượt học (tăng lên 50-100 nếu có GPU)
    BATCH_SIZE = 64 # Số mẫu học trong 1 lần
    
    history = model.fit(
        X_train, y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(X_val, y_val),
        callbacks=[
            # Dừng sớm nếu val_loss không cải thiện
            keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
        ]
    )
    
    print("Hoàn tất huấn luyện!")
    
    # Lưu mô hình (tùy chọn)
    model.save('lstm_model.h5')
    print("Đã lưu mô hình vào 'lstm_model.h5'")

else:
    print("Lỗi: Mô hình chưa được định nghĩa.")

# %%

In [ ]:

# --- Cell 10: Đánh giá Loss và "Accuracy" ---

if 'history' in locals():
    print("--- Đánh giá kết quả huấn luyện ---")

    # 1. Trực quan hóa Loss (MSE) và MAE
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))
    
    # Plot Loss (MSE)
    ax1.plot(history.history['loss'], label='Train Loss (MSE)')
    ax1.plot(history.history['val_loss'], label='Validation Loss (MSE)')
    ax1.set_title('Model Loss (Mean Squared Error)')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss (MSE)')
    ax1.legend()
    ax1.grid(True)
    
    # Plot "Accuracy" (Chúng ta dùng MAE - Mean Absolute Error)
    # GHI CHÚ: "Accuracy" là metric cho bài toán Phân loại (Classification).
    # Đối với bài toán Hồi quy (Regression) như dự đoán KPI, 
    # chúng ta dùng MAE để đo "độ chính xác" (sai số tuyệt đối trung bình).
    
    ax2.plot(history.history['mae'], label='Train MAE')
    ax2.plot(history.history['val_mae'], label='Validation MAE')
    ax2.set_title('Model "Accuracy" (Mean Absolute Error)')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Mean Absolute Error (MAE)')
    ax2.legend()
    ax2.grid(True)
    
    plt.show()

    # 2. Đánh giá trên tập TEST (Dữ liệu chưa từng thấy)
    print("\n--- Đánh giá trên tập Test ---")
    # model.evaluate sẽ trả về [loss, metric_1, metric_2, ...]
    # Tương ứng với compile(loss='mse', metrics=['mae'])
    test_results = model.evaluate(X_test, y_test, verbose=0)
    
    test_loss_mse = test_results[0]
    test_metric_mae = test_results[1]

    print(f"  Test Loss (MSE): {test_loss_mse:.6f}")
    print(f"  Test MAE (Mean Absolute Error): {test_metric_mae:.6f}")
    
    print("\nGiải thích MAE:")
    print(f"Giá trị MAE = {test_metric_mae:.6f} (trên dữ liệu đã scale từ 0-1).")
    print("Điều này có nghĩa là, trên thang 0-1, dự đoán của mô hình")
    print(f"sai lệch trung bình khoảng {test_metric_mae:.6f} so với giá trị thực tế.")
    print("(Để có MAE theo đơn vị gốc (MB, Users...), chúng ta cần dùng scaler.inverse_transform)")
    
else:
    print("Lỗi: Biến 'history' không tồn tại. Mô hình chưa được huấn luyện.")

In [ ]:
# --- Cell 11: So sánh kết quả dự đoán với giá trị thực tế ---

if 'model' in locals() and 'X_test' in locals() and 'y_test' in locals() and 'scaler' in locals():
    print("--- Trực quan hóa kết quả dự đoán trên tập Test ---")

    # 1. Dự đoán trên tập Test
    # Chọn một batch từ X_test để dự đoán
    # (Để đơn giản, ta lấy một vài mẫu đầu tiên hoặc một mẫu ngẫu nhiên)
    num_samples_to_plot = min(5, X_test.shape[0]) # Vẽ 5 mẫu hoặc ít hơn nếu X_test ít hơn

    if num_samples_to_plot == 0:
        print("Không có mẫu nào trong X_test để dự đoán và trực quan hóa.")
    else:
        # Lấy các mẫu đầu tiên từ X_test và y_test
        X_test_subset = X_test[:num_samples_to_plot]
        y_test_subset = y_test[:num_samples_to_plot]

        # Thực hiện dự đoán
        y_pred_scaled = model.predict(X_test_subset)

        # Đảo ngược quá trình chuẩn hóa (Inverse Scaling) cho cả giá trị thực và dự đoán
        # Chúng ta cần reshape lại dữ liệu thành 2D trước khi inverse_transform
        
        # Reshape y_test_subset (batch_size, output_steps, n_features) -> (batch_size * output_steps, n_features)
        y_true_reshaped = y_test_subset.reshape(-1, N_FEATURES)
        y_true_original = scaler.inverse_transform(y_true_reshaped)
        y_true_original = y_true_original.reshape(num_samples_to_plot, OUTPUT_STEPS, N_FEATURES)

        # Reshape y_pred_scaled (batch_size, output_steps, n_features) -> (batch_size * output_steps, n_features)
        y_pred_reshaped = y_pred_scaled.reshape(-1, N_FEATURES)
        y_pred_original = scaler.inverse_transform(y_pred_reshaped)
        y_pred_original = y_pred_original.reshape(num_samples_to_plot, OUTPUT_STEPS, N_FEATURES)

        # 2. Vẽ biểu đồ so sánh cho từng mẫu
        for i in range(num_samples_to_plot):
            print(f"\n--- So sánh Mẫu #{i+1} ---")
            
            fig, axes = plt.subplots(N_FEATURES, 1, figsize=(15, 5 * N_FEATURES), sharex=True)
            if N_FEATURES == 1: # Xử lý trường hợp chỉ có 1 feature (axes sẽ không phải mảng)
                axes = [axes]

            for j, feature_name in enumerate(FEATURE_COLS):
                ax = axes[j]
                ax.plot(y_true_original[i, :, j], label=f'Thực tế ({feature_name})', color='blue', marker='o', linestyle='-')
                ax.plot(y_pred_original[i, :, j], label=f'Dự đoán ({feature_name})', color='red', marker='x', linestyle='--')
                ax.set_title(f'Dự đoán {feature_name} (Mẫu #{i+1})')
                ax.set_xlabel('Bước thời gian (15 phút)')
                ax.set_ylabel(f'Giá trị {feature_name}')
                ax.legend()
                ax.grid(True)
            
            plt.tight_layout()
            plt.show()

        print("\nHoàn tất trực quan hóa.")

else:
    print("Lỗi: Không thể trực quan hóa. Đảm bảo 'model', 'X_test', 'y_test', 'scaler' đã tồn tại và X_test không rỗng.")